In [ ]:
import geopandas as geo
import matplotlib.pyplot as plt
import numpy as np
import pandas as pn

In [ ]:
#Format lith for export to SKS:
#overall model dimensions:         x: 251  y: 200,  z: 100
#surflist: 1D array of lithology values at land surface (for a 2D geologic model)

def export2gslib(export_filename, lith_array, grid_info):
    '''Export a lithology array to a gslib format that can be used with SKS (GeoModeller format).
    Inputs: 
    export_filename: name string of file to create (ex: 'surfgeo.csv')
    lith_array:      1D array of lithology values (for example, surflist returned by get_surflith()). Should have dimensions (xres*yres)
    grid_info:       [xmin,xmax,xres,dx,ymin,ymax,yres,dy,zmin,zmax,zres,dz] array of model grid and resolution info from importDEM()   
    
    Outputs:
    'geo.gslib':     csv gslib file of dimensions (2,xres*yres) with lithology integer values for each cell, formatted for SKS (GeoModeller format)
    '''
    
    #Get grid info:
    xres = grid_info[2]
    yres = grid_info[6]
    
    g = np.reshape(lith_array,(yres,xres)) #reshape 1D lith array to an array with dimensions (yres,xres)
    #print(np.shape(g))                     #print shape to check that reshape worked correctly (should be (xres,yres))
    g = np.flip(g,axis=0)                  #flip y axis 
    g = np.reshape(g,yres*xres)            #reshape 2D array back to 1D array
    #print(np.shape(g))                     #print shape to check that reshape worked correctly (should be (xres*yres))
    df = pn.DataFrame(g)                   #store in a pandas dataframe
    df = np.round(df)                      #round formation values to closest integer
    df = df.astype(np.int)                 #convert floats to integers
    header = pn.DataFrame(['Gottesacker geologic model from GemPy',1,'lith']) #set gslib file header
    df = header.append(df)                                   #attach header and data
    df.to_csv(export_filename, header=False, index=False)    #write the text file in gslib format

export_filename = 'surfgeo.gslib'
lith_array = surflist
export2gslib(export_filename, lith_array, grid_info)

In [ ]:
df = pn.read_csv('')

In [ ]:
import geopandas as geo

a = geo.read_file('Export_Output.shx', enabled_drivers=['GeoJSON', 'ESRI Shapefile']) #import shapefile (must be .shx not .shp)
im = a['GRID_CODE'].values.reshape(300,380)   #reshape data from x,y,z to a grid
plt.imshow(im, extent=(a['x'].min(), a['x'].max(), a['y'].min(), a['y'].max() )) #plot raster as image


In [ ]:
b = (a['x'].max() - a['x'].min())/50      #calculate number of cols
c = (a['y'].max() - a['y'].min())/50     #calculate number of rows
print('x:',b,' y:',c)

In [ ]:
im = a['GRID_CODE'].values.reshape(300,380)   #reshape data from x,y,z to a grid

In [ ]:
a['x'].min(), a['x'].max(), a['y'].min(), a['y'].max(), a['GRID_CODE'].max()

In [ ]:
#make interactive (magic):
%matplotlib notebook  
#%matplotlib inline                   

plt.imshow(im, extent=(a['x'].min(), a['x'].max(), a['y'].min(), a['y'].max() )) #plot raster as image
plt.show()

In [ ]:
import bokeh
bokeh.io.output_notebook()

In [ ]:
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Column
from bokeh.io import curdoc
from bokeh.events import DoubleTap

coordList=[]

TOOLS = "tap,reset"
bound = 10
p = figure(title='Double click to leave a dot.',
           tools=TOOLS,width=700,height=700,
           x_range=(-bound, bound), y_range=(-bound, bound))

source = ColumnDataSource(data=dict(x=[], y=[]))   
p.circle(source=source,x='x',y='y') 

#add a dot where the click happened
def callback(event):
    Coords=(event.x,event.y)
    coordList.append(Coords) 
    source.data = dict(x=[i[0] for i in coordList], y=[i[1] for i in coordList])        
p.on_event(DoubleTap, callback)

layout=Column(p)

curdoc().add_root(layout)

In [ ]:
bokeh.plotting.show(p)

In [ ]:
#Changing subplot sizes
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# create figure
f, ax = plt.subplots(3, 1, figsize=(10,10))

# plot some data
ax[0].plot([1, 2, 3])
ax[1].plot([1, 0, 1])
ax[2].plot([1, 2, 20])

# adjust subplot sizes
gs = GridSpec(3, 1, height_ratios=[5, 2, 1])
for i in range(3):
    ax[i].set_position(gs[i].get_position(f))

plt.show()

In [26]:
#Calling matlab scripts (simple version):
import subprocess

program  = 'C:/Program Files/MATLAB/R2017b/bin/matlab.exe'    #path to MATLAB exe
folder   = 'C:/Users/Chloe/Desktop/PhD/SKS/'                  #path to MATLAB folder with scripts to run
script   = 'triangle_area'                                    #name of script to run

#Variables
#Define variables to feed to function
b = 5 
h = 3
args = '({},{})'.format(b,h)  #put all args into one string

#Function string:
#Structure:  """path_to_exe -r "cd(fullfile('path_to_folder')), script(args)" """
#Example:    """C:/Program Files/MATLAB/R2017b/bin/matlab.exe -r "cd(fullfile('C:/Users/Chloe/Desktop/PhD/SKS/')), triangle_area(5,3)" """
fun =  """{} -r "cd(fullfile('{}')), {}{}" """.format(program, folder, script, args)  #create function string that tells subprocess what to do
subprocess.Popen(fun)


C:/Program Files/MATLAB/R2017b/bin/matlab.exe -r "cd(fullfile('C:/Users/Chloe/Desktop/PhD/SKS/')), triangle_area(10,5)" 


In [42]:
#Calling matlab scripts (no arguments, suppress window):
import subprocess as sp

program  = 'C:/Program Files/MATLAB/R2017b/bin/matlab.exe'    #path to MATLAB exe
folder   = 'C:/Users/Chloe/Desktop/PhD/SKS/'                  #path to MATLAB folder with scripts to run
script   = 'triangle_area2'                                   #name of script to run
options  = '-nosplash -nodesktop -wait'                       #optional: set run options  (nosplash? nodesktop means MATLAB won't open a new desktop window, wait means Python will wait until MATLAB is done beore continuing (needs to be paired with p.wait() after sp.Popen))
has_args = True                                               #does the MATLAB script need arguments?

#Optional: define arguments to feed to function
if has_args ==True:
    script   = 'triangle_area'                                #version with arguments
    b = 5 
    h = 3
    args = '({},{})'.format(b,h)  #put all args into one string

#Set function string:
#Structure:  """path_to_exe optional_arguments -r "cd(fullfile('path_to_folder')), script_name, exit" """
#Example:    """C:/Program Files/MATLAB/R2017b/bin/matlab.exe -r "cd(fullfile('C:/Users/Chloe/Desktop/PhD/SKS/')), triangle_area, exit" """
#basically, needs to know where the program to use lives, then takes some optional settings, -r runs the program, cd changes to the directory with the script, then needs the name of the script (possibly with arguments), then exits
fun =  """{} {} -r "cd(fullfile('{}')), {}, exit" """.format(program, options, folder, script)  #create function string that tells subprocess what to do
if has_args==True:
    fun = """{} {} -r "cd(fullfile('{}')), {}{}, exit" """.format(program, options, folder, script, args)
print('command:', fun)

#Run MATLAB:
print('running MATLAB script...')
p = sp.Popen(fun)                     #open the subprocess & run the MATLAB script 
p.wait()                              #wait until MATLAB is done before proceeding (this needs to be paired with -wait in options?)
print('done')                 

command: C:/Program Files/MATLAB/R2017b/bin/matlab.exe -nosplash -nodesktop -wait -r "cd(fullfile('C:/Users/Chloe/Desktop/PhD/SKS/')), triangle_area(5,3), exit" 
running MATLAB script...
done
